In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("Datos_icfes_limpios.csv")

In [3]:
threshold = df['punt_global'].quantile(0.25)
df['bajo_desempeno'] = (df['punt_global'] <= threshold).astype(int)

In [4]:
cols_utiles = [col for col in df.columns if (
    col.startswith('fami_') or
    (col.startswith('cole_bilingue')) or
    col.startswith('estu_genero')
)]
X = df[cols_utiles].copy()
y = df['bajo_desempeno']

In [5]:
X = X.apply(pd.to_numeric, errors='coerce')
X = X.dropna(axis=1, how='all')
X = X.dropna()
y = y.loc[X.index]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(weights))


In [8]:
print("Columnas usadas en X:", X.columns.tolist())
print("Forma de X:", X.shape)

Columnas usadas en X: ['cole_bilingue-N', 'cole_bilingue-S', 'estu_genero-F', 'estu_genero-M', 'fami_cuartoshogar-Cinco', 'fami_cuartoshogar-Cuatro', 'fami_cuartoshogar-Dos', 'fami_cuartoshogar-Seis o mas', 'fami_cuartoshogar-Tres', 'fami_cuartoshogar-Uno', 'fami_educacionmadre-Educación profesional completa', 'fami_educacionmadre-Educación profesional incompleta', 'fami_educacionmadre-Ninguno', 'fami_educacionmadre-Postgrado', 'fami_educacionmadre-Primaria completa', 'fami_educacionmadre-Primaria incompleta', 'fami_educacionmadre-Secundaria (Bachillerato) completa', 'fami_educacionmadre-Secundaria (Bachillerato) incompleta', 'fami_educacionmadre-Técnica o tecnológica completa', 'fami_educacionmadre-Técnica o tecnológica incompleta', 'fami_educacionpadre-Educación profesional completa', 'fami_educacionpadre-Educación profesional incompleta', 'fami_educacionpadre-Ninguno', 'fami_educacionpadre-Postgrado', 'fami_educacionpadre-Primaria completa', 'fami_educacionpadre-Primaria incompleta'

In [9]:
model = Sequential([
    Dense(256, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])


c:\Users\maria\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
import mlflow
import mlflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Hiperparámetros a probar
neuronas_list = [64, 128, 256]
dropouts = [0.2, 0.3]
learning_rates = [0.001, 0.0005]

# Crear experimento si no existe
mlflow.set_experiment("experimento_red_neuronal_icfes")

# Research loop
for n in neuronas_list:
    for d in dropouts:
        for lr in learning_rates:
            with mlflow.start_run():
                # 🧾 Log de hiperparámetros
                mlflow.log_param("neuronas", n)
                mlflow.log_param("dropout", d)
                mlflow.log_param("learning_rate", lr)

                # 🧠 Modelo
                model = Sequential([
                    Dense(n, input_dim=X_train_scaled.shape[1], activation='relu'),
                    Dropout(d),
                    Dense(n//2, activation='relu'),
                    Dense(1, activation='sigmoid')
                ])
                model.compile(
                    loss='binary_crossentropy',
                    optimizer=Adam(learning_rate=lr),
                    metrics=['accuracy']
                )

                # 🏋️‍♂️ Entrenamiento
                history = model.fit(X_train_scaled, y_train,
                                    validation_split=0.2,
                                    epochs=10,
                                    batch_size=32,
                                    verbose=0)

                # 📊 Evaluación
                loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)

                # 🧪 Registro de métricas
                mlflow.log_metric("loss", loss)
                mlflow.log_metric("accuracy", accuracy)

                # 💾 Registro del modelo (con ejemplo de entrada)
                import numpy as np
                mlflow.keras.log_model(model, "modelo_keras")



2025/05/20 20:57:39 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/05/20 20:58:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 21:06:01 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/05/20 21:06:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 21:16:31 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/05/20 21:16:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/20 21:26:51 WARNING mlflow.keras.save: You are saving a Keras model 

In [10]:
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [14]:
history = model.fit(X_train_scaled, y_train,
          epochs=20,
          batch_size=32,
          validation_split=0.2,
          class_weight=class_weights,
          verbose=1)



Epoch 1/20
10248/10248 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - accuracy: 0.6532 - loss: 0.6171 - val_accuracy: 0.6379 - val_loss: 0.6002
Epoch 2/20
10248/10248 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.6294 - loss: 0.6129 - val_accuracy: 0.6307 - val_loss: 0.6113
Epoch 3/20
10248/10248 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.6343 - loss: 0.6120 - val_accuracy: 0.6014 - val_loss: 0.6346
Epoch 4/20
10248/10248 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.6365 - loss: 0.6103 - val_accuracy: 0.6425 - val_loss: 0.6001
Epoch 5/20
10248/10248 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.6380 - loss: 0.6110 - val_accuracy: 0.6125 - val_loss: 0.6265
Epoch 6/20
10248/10248 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.6366 - loss: 0.6104 - val_accuracy: 0.6328 - val_loss: 0.6167
Epoch 7/20
10248/10248 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.6357 - loss: 0.6115 - val_accuracy: 0.6436 - val_loss: 0.6122
Epoch 8/20
10248/10248 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 

In [1]:
y_pred_prob = model.predict(X_test_scaled).ravel()
y_pred = (y_pred_prob >= 0.35).astype(int)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("ROC AUC Score:")
print(roc_auc_score(y_test, y_pred_prob))


NameError: name 'model' is not defined